In [3]:
from numpy import log10, pi
from numpy.linalg import norm
import numpy as np
from scipy.constants import c
from scipy.stats import multivariate_normal
import scipy.optimize as opt
import fileinput
import time
import glob
import json
import datetime
import itertools
import pandas as pd
import matplotlib.pyplot as plt

f = 2.4e9
X_OFFSET = -150
Y_OFFSET = -150
Z_OFFSET = 0
CELL_SIZE = 10
GRID_MATRIX_SIZE = (30, 35)
NORM_STD = [[10, 0], [0, 10]]

def _create_args(sensors, sigma):
    return (sensors, sigma)

def _get_args(args):
    """
        Returns elements from args created by create_args
        RETURN (sensors, sigma)
    """
    return args[0], args[1]

def _friis(sensor, x, y, Pt=1, z=2):
    return (Pt + 20 * log10(c / (4 * pi * f)) -
            10 * log10((x - sensor['x'])**2 + (y - sensor['y'])**2 + z**2))

def _loss(pos, *args):
    sensors, sigma = _get_args(args)
    return sum([(sensor['signal'] - _friis(sensor, pos[0], pos[1]))**2 / sigma**2 for sensor in sensors])

def read_data(ids=[]):
    start = time.time()
    data = {}
    files = glob.glob('/media/nickyz/Data/scriptie_data/armin/20170512_[5].csv')
    print(files)
    files.reverse()

    for line in fileinput.input(files=files):
        if fileinput.isfirstline():
            print(time.time() - start)
            print(fileinput.filename())
            start = time.time()
            continue

        splitted = line.split('\t')
        values = json.loads(splitted[0])

        try:
            sourceMac = values['sourceMac']
        except KeyError:
            sourceMac = values['sourcemac']

        if ids != [] and sourceMac not in ids:
            continue
        
        try:
            localMac = int(values['localMac'])
        except KeyError:
            localMac = int(values['localmac'])

        signal = int(values['signal'])

        if localMac:
            continue

        try:
            droneId = values['droneId']
        except KeyError:
            droneId = values['droneid']

        timestamp = splitted[2]
        orig_time = datetime.datetime.fromtimestamp(int(timestamp[0:-3])).strftime('%Y-%m-%d %H:%M:%S')
        timestamp = datetime.datetime.fromtimestamp(int(timestamp[0:-3])).strftime('%Y-%m-%d %H:%M')

        try:
            data[sourceMac].append([timestamp, droneId, signal, orig_time])
        except KeyError:
            data[sourceMac] = [[timestamp, droneId, signal, orig_time]]

    print(time.time() - start)

    return data

def read_toppers_data(ids=[]):
    """
        0 -> sourcemac
        1 -> sensorid
        2 -> typenr
        3 -> signal
        4 -> localmac
        5 -> seqnr
        6 -> retryflag
        7 -> subtypenr
        8 -> coordx
        9 -> coordy
        10 -> type
        11 -> source
        12 -> version
        13 -> measurementtimestamp
        14 -> processingtimestamp
        15 -> application
        16 -> date
    """
    start = time.time()
    data = {}
    files = glob.glob('/media/nickyz/Data/scriptie_data/u2/20170729_[2-4].csv')
    print(files)
    files.reverse()

    for line in fileinput.input(files=files):
        if fileinput.isfirstline():
            print(time.time() - start)
            print(fileinput.filename())
            start = time.time()
            continue

        splitted = line.split('\t')

        sourceMac = splitted[0]
        localMac = int(splitted[4])
        signal = int(splitted[3])

        if localMac:
            continue
        
        if ids != [] and sourceMac not in ids:
            continue
            
        droneId = splitted[1]

        timestamp = splitted[13]
        orig_time = datetime.datetime.fromtimestamp(int(timestamp[0:-3])).strftime('%Y-%m-%d %H:%M:%S')
        timestamp = datetime.datetime.fromtimestamp(int(timestamp[0:-3])).strftime('%Y-%m-%d %H:%M')

        try:
            data[sourceMac].append([timestamp, droneId, signal, orig_time])
        except KeyError:
            data[sourceMac] = [[timestamp, droneId, signal, orig_time]]

    print(time.time() - start)

    return data

def read_drone_data():
    drone_data = pd.read_csv('/media/nickyz/Data/scriptie_data/Huawei_routers_locations_at_Arena/sensors_arena_sensation.csv')

    return drone_data

def find_location(drone_id, drone_data):
    drone = drone_data.loc[drone_data['drone_id'] == drone_id]

    try:
        return drone.iloc[0]
    except IndexError:
        return None

def group_data(data, sort_index):
    sorted_data = sorted(data, key=lambda x: x[sort_index])

    return itertools.groupby(sorted_data, lambda x: x[sort_index])

def grid_index(loc, bottom, size=10):
    return int((loc[0] - bottom[0]) / size), int((loc[1] - bottom[1]) / size), int((loc[2] - bottom[2]) / size)

def top_loc(index_count):
    return max(index_count, key=lambda x: index_count[x])

def get_xy(index, bottom, size=10):
    return (bottom[0] + (index[0] + 0.5) * size, bottom[1] + (index[1] + 0.5) * size)

def create_norm_matrix(mean):
    x_space = np.linspace(0, GRID_MATRIX_SIZE[1], GRID_MATRIX_SIZE[1], endpoint=False)
    y_space = np.linspace(0, GRID_MATRIX_SIZE[0], GRID_MATRIX_SIZE[0], endpoint=False)
    X, Y = np.meshgrid(x_space, y_space)
    pos = np.empty(X.shape + (2,))
    pos[:, :, 0] = X; pos[:, :, 1] = Y
    
    rv = multivariate_normal(mean, NORM_STD)
    return rv.pdf(pos)

def get_max_matrix_value(matrix):
    max_index = matrix.argmax()
    
    return (max_index % GRID_MATRIX_SIZE[1], max_index // GRID_MATRIX_SIZE[1])
        

In [5]:
data = read_data(["36f8ef8e-d6aa-4ac2-91f8-b6f2779a2974"])
drone_data = read_drone_data()

['/media/nickyz/Data/scriptie_data/armin/20170512_5.csv']
0.14048480987548828
/media/nickyz/Data/scriptie_data/armin/20170512_5.csv
24.309646129608154


In [25]:
results = []

for drone_id in data:
    time_grouped = group_data(data[drone_id], 0)
    for time_group in time_grouped:
        values = list(time_group[1])
        g_data = [(item[0], item[1], item[2], item[3]) for item in values]
        g_data = sorted(g_data, key=lambda x: -x[2])
        
        index_count = {}
        ids = []
        all_locs = []
        
        for row in g_data:
            loc = find_location(row[1], drone_data)
            
            if loc is None:
                continue

            loc_tup = (loc['x_m'], loc['y_m'], loc['z_m'])
            
            if row[1] not in ids:
                ids.append(row[1])
                all_locs.append({'signal': float(row[2]), 'x': loc['x_m'], 'y': loc['y_m']})
            
            index = grid_index(loc_tup, (X_OFFSET, Y_OFFSET, Z_OFFSET), CELL_SIZE)
            index_tup = (index[0], index[1])
            try:
                index_count[index_tup] += 1
            except KeyError:
                index_count[index_tup] = 1

        res = get_xy(top_loc(index_count), (X_OFFSET, Y_OFFSET, Z_OFFSET), CELL_SIZE)
        results.append([res[0], res[1], all_locs])

In [6]:
results = []

for drone_id in data:
    prev_pos = None

    time_grouped = group_data(data[drone_id], 0)
    for time_group in time_grouped:
        values = list(time_group[1])
        g_data = [(item[0], item[1], item[2], item[3]) for item in values]
        
        index_counts = np.zeros(GRID_MATRIX_SIZE)
        ids = []
        all_locs = []
        
        for row in g_data:
            loc = find_location(row[1], drone_data)
            
            if loc is None:
                continue

            loc_tup = (loc['x_m'], loc['y_m'], loc['z_m'])
            
            if row[1] not in ids:
                ids.append(row[1])
                all_locs.append({'signal': float(row[2]), 'x': loc['x_m'], 'y': loc['y_m']})

            index = grid_index(loc_tup, (X_OFFSET, Y_OFFSET, Z_OFFSET), CELL_SIZE)[:2]
            index_counts[index[1], index[0]] += 1
        
        if prev_pos is None:
            prev_pos = get_max_matrix_value(index_counts)
        else:
            norm_matrix = create_norm_matrix((prev_pos[1], prev_pos[0]))
#             print(np.array([row for row in norm_matrix])[3:12, 17:26])
#             print(prev_pos)
            prev_pos = get_max_matrix_value(np.einsum('ij,ij->ij', index_counts, norm_matrix))

        res = get_xy(prev_pos, (X_OFFSET, Y_OFFSET, Z_OFFSET), CELL_SIZE)
        results.append([res[0], res[1], all_locs])

In [21]:
print(results)

[[75.0, -65.0, [{'y': 10.642000000000001, 'signal': -83.0, 'x': -117.807}, {'y': 69.340000000000003, 'signal': -74.0, 'x': -66.912999999999997}, {'y': 70.292000000000002, 'signal': -74.0, 'x': -60.796000000000006}, {'y': 66.292000000000002, 'signal': -73.0, 'x': -71.484999999999999}, {'y': 30.186999999999998, 'signal': -88.0, 'x': -100.90600000000001}, {'y': 52.168999999999997, 'signal': -90.0, 'x': -55.451999999999998}, {'y': 60.292000000000002, 'signal': -89.0, 'x': -81.105000000000004}, {'y': 79.540999999999997, 'signal': -76.0, 'x': -61.233999999999995}, {'y': 50.884999999999998, 'signal': -89.0, 'x': -89.835999999999999}, {'y': 57.678999999999995, 'signal': -78.0, 'x': -94.634}, {'y': 63.721000000000004, 'signal': -79.0, 'x': -77.421999999999997}, {'y': 31.245000000000001, 'signal': -80.0, 'x': -129.39500000000001}, {'y': 67.054000000000002, 'signal': -78.0, 'x': -72.533000000000001}, {'y': 0.72699999999999998, 'signal': -89.0, 'x': -93.932999999999993}, {'y': 10.642000000000001, 

In [7]:
xs = [row['x_m'] for i, row in drone_data.iterrows() if row['x_m'] < 1000]
ys = [row['y_m'] for i, row in drone_data.iterrows() if row['y_m'] < 1000]

for res in results:
    for loc in res[2]:
        plt.scatter(loc['x'], loc['y'], s=150, alpha=0.3)
    plt.scatter(xs, ys)
    plt.scatter(res[0], res[1], color='r')
    plt.xticks(np.arange(-150, 200, CELL_SIZE))
    plt.yticks(np.arange(-150, 150, CELL_SIZE))
    plt.grid()
    plt.show()

/usr/lib/python3/dist-packages/matplotlib/collections.py:549: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


KeyboardInterrupt: 